In [5]:
import pandas as pd
from autocorrect import Speller
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import re
import pickle
from statistics import fmean

In [9]:
lemmatizer = WordNetLemmatizer()
spell = Speller(lang='en')
word2vec_model = gensim.models.Word2Vec.load('word2vec.model')
stop_words = stopwords.words('english')

In [19]:
train = pd.read_csv('data/rec_train.csv')

In [22]:
test = pd.read_csv('data/rec_test.csv')

In [ ]:
with open('freq.pkl', 'rb') as f:
    freq = pickle.load(f)

In [1]:
def get_recommendation(description, data, freq):
    reviews_lower = [description.lower()]
    cleaned_reviews = [re.sub('[^a-zA-Z-]', ' ', x) for x in reviews_lower]
    cleaned_reviews = [re.sub(r'\s+', ' ', x) for x in cleaned_reviews]
    reviews = [nltk.sent_tokenize(x) for x in cleaned_reviews]
    reviews = [y for x in reviews for y in x]
    words = [nltk.word_tokenize(x) for x in reviews]
    for i in range(len(words)):
        words[i] = [lemmatizer.lemmatize(w) for w in words[i] if w not in stop_words and len(lemmatizer.lemmatize(w)) > 1]
    
    desc_score = []
    vocab = set(word2vec_model.wv.key_to_index.keys())
    for desc in words:
        word_score = []
        for w in desc:
            if w in vocab and w in freq:
                word_score.append(fmean(word2vec_model.wv[w] * freq[w]))
                # word_score.append(fmean(word2vec_model.wv[w]))
            elif spell(w) in vocab and spell(w) in freq:
                word_score.append(fmean(word2vec_model.wv[spell(w)] * freq[spell(w)]))
        desc_score.append(fmean(word_score))
    
    recommendations = []
    for s in desc_score:
        recommendations.append(data.iloc[(data['desc_score']-s).abs().argsort()[:1]])
    
    return recommendations

In [24]:
train.iloc[(train['desc_score']-test.iloc[0].desc_score).abs().argsort()[:2]]

,country,description,designation,points,price,province,region_1,region_2,variety,winery,taster_name,taster_twitter_handle,title,desc_score
57651,US,"This is straightforward and easy, with sweet M...",NaN,85,16.0,California,Napa Valley,Napa,Sauvignon Blanc,Lola Kay,NaN,NaN,NaN,-0.048324
81972,US,"This is a lovely, full-bodied expression of ap...",Reserve,91,35.0,California,Napa Valley,Napa,Chardonnay,Stonehedge,Virginie Boone,@vboone,Stonehedge 2014 Reserve Chardonnay (Napa Valley),-0.048321
